In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# ---------------------------------------
# 1. Data Loading
# ---------------------------------------

def load_and_prepare_data():
    macro = pd.read_csv("macro_data.csv", parse_dates=["DATE"]).set_index("DATE")
    forex = pd.read_csv("forex_merged_cleaned.csv", parse_dates=["DATE"]).set_index("DATE")
    
    log_returns = np.log(forex / forex.shift(1)).dropna()
    log_returns.columns = [col + " Return" for col in log_returns.columns]

    LAG_PERIODS = (1, 2, 3, 4, 5, 12, 24, 60)
    ROLL_WINDOWS = (3, 5, 7, 10)

    def enrich_macro_safe(df):
        fea = df.copy()
        for p in LAG_PERIODS:
            fea = pd.concat([fea, df.shift(p).add_suffix(f"_lag{p}")], axis=1)
        for w in ROLL_WINDOWS:
            fea = pd.concat([
                fea,
                df.rolling(w).mean().shift(1).add_suffix(f"_rollmean{w}"),
                df.rolling(w).std().shift(1).add_suffix(f"_rollstd{w}")
            ], axis=1)
        return fea.dropna()

    macro_fea = enrich_macro_safe(macro)
    
    full_df = pd.merge(macro_fea, log_returns, left_index=True, right_index=True).dropna()
    return full_df, forex

# ---------------------------------------
# 2. Model Training Helper
# ---------------------------------------

def train_model(X_train, y_train, selected_model):
    if selected_model == 'Lasso':
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        
        model = Lasso(alpha=0.01, max_iter=10000)
        model.fit(X_train_scaled, y_train)
        
        return model, scaler
    elif selected_model == 'XGBoost':
        model = XGBRegressor(n_estimators=500, learning_rate=0.05)
        model.fit(X_train, y_train)
        
        return model, None
    else:
        raise ValueError(f"Model {selected_model} not supported.")

# ---------------------------------------
# 3. Full Main Function
# ---------------------------------------

def run_forex_model(selected_macros, selected_model, macro_adjustments=None, future_years=10):
    """
    selected_macros: list of macro factors like ['Interest Rate', 'Inflation']
    selected_model: string, either 'Lasso' or 'XGBoost'
    macro_adjustments: dictionary like {'Interest Rate': 0.5} (optional)
    future_years: 5, 7, or 10 years
    """

    full_df, forex = load_and_prepare_data()
    
    fx_targets = [col for col in full_df.columns if "Return" in col]
    available_macros = [col for col in full_df.columns if any(macro in col for macro in selected_macros)]

    # Train/Test split
    train_end = "2022-12-31"
    train_df = full_df[full_df.index <= train_end]
    test_df = full_df[full_df.index > train_end]

    all_results = []
    all_future_preds = []

    for target in fx_targets:
        y_train = train_df[target]
        y_test = test_df[target]
        X_train = train_df[available_macros]
        X_test = test_df[available_macros]
        
        model, scaler = train_model(X_train, y_train, selected_model)

        # --- Predict on real 2023-2025 data ---
        if scaler:
            preds = model.predict(scaler.transform(X_test))
        else:
            preds = model.predict(X_test)

        r2 = r2_score(y_test, preds)
        mae = mean_absolute_error(y_test, preds)
        rmse = np.sqrt(mean_squared_error(y_test, preds))

        all_results.append({
            "Currency": target.replace(" Return", ""),
            "Model": selected_model,
            "Selected_Macros": selected_macros,
            "R2_Score": r2,
            "MAE": mae,
            "RMSE": rmse,
        })

        # --- Now predict FUTURE based on slider-adjusted macros ---
        last_known_macros = train_df[available_macros].iloc[-1]
        
        # Create future macro frame (same adjustment across months)
        months = future_years * 12
        future_macro_df = pd.DataFrame([last_known_macros.values] * months, columns=available_macros)

        if macro_adjustments:
            for macro, adjustment in macro_adjustments.items():
                matching_cols = [col for col in available_macros if macro in col]
                for col in matching_cols:
                    future_macro_df[col] += adjustment
        
        # Predict future
        if scaler:
            future_macro_scaled = scaler.transform(future_macro_df)
            future_preds = model.predict(future_macro_scaled)
        else:
            future_preds = model.predict(future_macro_df)
        
        # Rebuild Forex Prices
        forex_pair = target.replace(" Return", "")
        
        # Use last available actual price
        last_real_price = forex[forex_pair].iloc[-1]

        future_price_series = last_real_price * np.exp(pd.Series(future_preds)).cumprod()
        
        future_dates = pd.date_range(start="2025-01-01", periods=months, freq='MS')
        temp_future_df = pd.DataFrame({
            'DATE': future_dates,
            'Currency': forex_pair,
            'Predicted_Return': future_preds,
            'Predicted_Forex_Rate': future_price_series.values
        })
        
        all_future_preds.append(temp_future_df)

    results_df = pd.DataFrame(all_results)
    future_preds_df = pd.concat(all_future_preds)

    return results_df, future_preds_df


In [17]:
real_metrics, future_predictions = run_forex_model(
    selected_macros=['Interest Rate', 'Inflation'],
    selected_model='Lasso',
    macro_adjustments={'Interest Rate': 0.5},   # user slider
    future_years=7   # user selects 5, 7, or 10
)

print(real_metrics)        # Real world 2023–2025 test accuracy
print(future_predictions)  # Predicted future forex rates!


  Currency  Model             Selected_Macros  R2_Score       MAE      RMSE
0  USD-AUD  Lasso  [Interest Rate, Inflation] -0.009747  0.021536  0.026352
1  USD-CAD  Lasso  [Interest Rate, Inflation] -0.021359  0.013683  0.016418
2  USD-CHF  Lasso  [Interest Rate, Inflation] -0.000021  0.021142  0.023468
3  USD-CNY  Lasso  [Interest Rate, Inflation] -0.008005  0.012120  0.014785
4  USD-EUR  Lasso  [Interest Rate, Inflation] -0.003284  0.016560  0.018896
5  USD-GBP  Lasso  [Interest Rate, Inflation] -0.036326  0.015806  0.019231
6  USD-HKD  Lasso  [Interest Rate, Inflation] -0.002343  0.001605  0.002216
7  USD-JPY  Lasso  [Interest Rate, Inflation] -0.012778  0.028693  0.033120
8  USD-NZD  Lasso  [Interest Rate, Inflation] -0.015653  0.024666  0.029770
9  USD-XAU  Lasso  [Interest Rate, Inflation] -0.110854  0.032001  0.038936
         DATE Currency  Predicted_Return  Predicted_Forex_Rate
0  2025-01-01  USD-AUD          0.000784              1.569366
1  2025-02-01  USD-AUD          0.0007